In [ ]:
from matplotlib import pyplot as plt
import matplotlib.patheffects as PathEffects
import matplotlib.image as mpimg

import numpy as np
import matplotlib

import numpy as np
import os
import json
from tqdm import tqdm 
import skvideo.io

path_data_dirname = "/media/giancos/Football/dataset_crop224/"
path_data_dirname_frame = "/media/giancos/Football/dataset/"

labels = ["background", "card", "substitution", "goal"]

gamePath = "england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/"
# gamePath = "england_epl/2014-2015/2015-05-17 - 18-00 Manchester United 1 - 1 Arsenal/"
# gamePath = "england_epl/2014-2015/2015-05-17 - 18-00 Manchester United 1 - 1 Arsenal/"
for gamePath in np.load(os.path.join(path_data_dirname, "listgame_test_100.npy")):

# for gamePath in np.load(os.path.join(path_data_dirname, "listgame.npy")):
# for gamePath in ["/media/giancos/Football/dataset_crop224/england_epl/2015-2016/2015-08-08 - 19-30 Chelsea 2 - 2 Swansea",
#                  "/media/giancos/Football/dataset_crop224/england_epl/2015-2016/2015-08-29 - 17-00 Manchester City 2 - 0 Watford"]:

    # for game in np.load(os.path.join(path_data_dirname, "listgame_test_100.npy")):
    #     print(game)

    print(os.path.join(path_data_dirname, gamePath))

    with open(os.path.join(path_data_dirname, gamePath, "Labels.json")) as file :
        JsonFile = json.loads(file.read())

        print(len(JsonFile['annotations']))




        fig, axs = plt.subplots(1,len(JsonFile['annotations']),num=None, 
                figsize=(2*len(JsonFile['annotations']), 3), 
                dpi=100, 
                facecolor='w',
                edgecolor='k',
                               sharey=True)


        #     plt.xlabel(r"Time", fontsize=30)
        # #     plt.ylabel(r"Score", fontsize=30)
        #     plt.tick_params(axis='both', which='major', labelsize=30)


        #     videogen = skvideo.io.vreader(videoPath, num_frames=int(game_end_index), backend='ffmpeg')

        half = 2
        for i, myEvent in enumerate(JsonFile['annotations']):
            if (half == 2 and int(myEvent['gameTime'][:1]) == 1):
                videogen = skvideo.io.vreader(os.path.join(path_data_dirname_frame, gamePath, "1.mkv"),  backend='ffmpeg')
                index_frame = 0
            elif (half == 1 and int(myEvent['gameTime'][:1]) == 2):
                videogen = skvideo.io.vreader(os.path.join(path_data_dirname_frame, gamePath, "2.mkv"),  backend='ffmpeg')
                index_frame = 0


            half = int(myEvent['gameTime'][:1])
            second = int(myEvent['gameTime'][-2:])
            minute = int(myEvent['gameTime'][-5:-3])
        #         print(json.dumps(myEvent,indent=4))

            t=minute*60+second     

            if (half == 1):
                predictions = np.load(os.path.join(path_data_dirname, gamePath, "Predictions_augm_50_Half_1.npy"))
                predictions = np.load(os.path.join(path_data_dirname, gamePath, "Predictions_Half_1.npy"))
            elif (half == 2):
                predictions = np.load(os.path.join(path_data_dirname, gamePath, "Predictions_augm_50_Half_2.npy"))
                predictions = np.load(os.path.join(path_data_dirname, gamePath, "Predictions_Half_2.npy"))


            frame_fileName = os.path.join(path_data_dirname_frame, gamePath,
                                          "Frame_%1d_%02d_%02d_" % (half, minute, second) +  myEvent['label'] + ".png")
            print(frame_fileName)
            if (os.path.exists(frame_fileName)):
                print("exist")
                
                arr_img = plt.imread(frame_fileName)
            
            else:
                print("NOT exist")
                    

                # try:
                for frame in videogen:

                    index_frame = index_frame + 1  
                    if (index_frame >= t*25.0):
                        arr_img = frame
                        skvideo.io.vwrite(frame_fileName, arr_img)
                        break

                    



            ax = axs[i]
            ax.set_axisbelow(True)
            ax.xaxis.grid(color='gray', linestyle='dashed')
            ax.yaxis.grid(color='gray', linestyle='dashed')


            ax.plot( [t,t], 
                    [0,1],
                    linewidth=4.0, color="k")

            labels= ["background", "card", "substitution", "goal"]
            c=["k", "b", "g", "r"]

            for i_label in [1,2,3]:
                i_init = np.max([t-60, 0]) 
                i_end = np.min([t+60, len(predictions[:,i_label])]) 
        #             print(i_init, i_end)
                ax.plot(np.arange(i_init,i_end), 
                        predictions[i_init:i_end,i_label],
                        linewidth=4.0, color=c[i_label], label=labels[i_label])

        #         legend = ax.legend(loc="upper left", shadow=True, frameon=True,
        #                            prop={'weight':'bold'}, handlelength=1.0,  )
        #         frame = legend.get_frame().set_facecolor('#FFFFFF')

        #         for label in legend.get_lines():
        #             label.set_linewidth(5)  # the legend line width

        #         for text in legend.get_texts():
        #             text.set_fontsize(25)


            ax.tick_params(axis='both', which='major', labelsize=20)
            plt.sca(ax)
            
            time_init = "%02d:%02d" % ((i_init+t)/2//60, (i_init+t)/2%60)
            time_end = "%02d:%02d" % ((i_end+t)/2//60, (i_end+t)/2%60)
            time_t = "%02d:%02d" % (t//60, t%60)
            
            locs, labels = plt.xticks([(i_init+t)/2, t, (i_end+t)/2], [time_init, time_t, time_end], rotation=60)


            


            from matplotlib.patches import Circle
            from matplotlib.offsetbox import (TextArea, DrawingArea, OffsetImage,
                                              AnnotationBbox)
            from matplotlib.cbook import get_sample_data


            imagebox = OffsetImage(arr_img, zoom=0.4)
            imagebox.image.axes = ax

            ab = AnnotationBbox(imagebox, (t,1),
                             xybox=(-0., 60.),
                            xycoords='data',
                            boxcoords="offset points",
                            pad=0.3,
                            arrowprops=dict(arrowstyle="->"))

            ax.add_artist(ab)

            
            ax.set_ylim(0, 3)
            plt.sca(ax)
            plt.yticks([0,0.5,1])


        plt.show()

#         path_figure = os.path.join(path_data_dirname, gamePath, "QualitativeResults.png")
#         fig.savefig(path_figure, bbox_inches='tight')




